# ДЗ 2. Content-based рекомендация

1. Приведите данные датасета `ratings` к виду датафрейма со строками-пользователями, столбцами-книгами и рейтингами на пересечении. Будем дополнять информацию о пользователе на основе прочитанных пользователем книг

(проверка: размерность датасета должна быть (53424, 10000) )

2. Создайте векторы, характеризующие пользоваталей (для content-based лучше это делать по дополнительным данным о пользователях, но так как тут у нас их нет - сделаем вектора-признаки (размерностью 100) на основе проставленных книгам оценок) 

Делать будем с помощью метода понижения размерности PCA до 100.

3. Объедините три датасета:  
    * ratings
    * pca_user_vectors(векторы-признаки для каждого пользователя)
    * tf-idf на основе `books['corpus']` для каждой книги
    
В итоге у вас должен получиться датафрейм с вектором пользователя, вектором книги и таргет-рейтинг.

4. Разбейте на train/valid set

5. Обучите любую модель машинного обучения (для задачи регрессии (линейную или ансамбль деревьев)), сделайте прогноз и посчитайте метрики RMSE, MSE
6. Добавьте другие признаки по книгам из `books`

7. Сделайте тестовый датасет для пользователей и книг, которые находятся в датасете to-read:
    * pca_user_vectors(векторы-признаки для каждого пользователя)
    * tf-idf на основе `books['corpus']` для каждой книги
8. Сделайте для них прогноз

## Загрузка данных из семинара

In [ ]:
!wget -O "input.zip" "https://www.dropbox.com/s/55ognewt1g3l09k/input.zip?dl=1" -q && unzip -o -q "input.zip" && rm "input.zip" && rm -r "__MACOSX"

In [ ]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix, save_npz, load_npz

from sklearn.decomposition import IncrementalPCA
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# dataframe of 'goodreads' books-shop
books = pd.read_csv('input/books.csv', encoding = "ISO-8859-1")
print(books.shape, books.isna().values.any(), books.duplicated().values.any())
print(books.columns)
books.sample(4)

(10000, 23) True False
Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
8443,8444,4946005,4946005,21899169,49,553807145,9.780554e+12,Dean Koontz,2009.0,Relentless,Relentless,en-US,3.71,12416,13662,1223,394,1016,4015,4940,3297,https://images.gr-assets.com/books/1320399029m...,https://images.gr-assets.com/books/1320399029s...
6849,6850,781498,781498,767519,18,140546235,9.780141e+12,"Trinka Hakes Noble, Steven Kellogg",1980.0,The Day Jimmy's Boa Ate the Wash,The Day Jimmy's Boa Ate the Wash,NaN,4.16,16875,17024,215,187,603,3333,5042,7859,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
4544,4545,13202092,13202092,18385042,34,547569653,9.780548e+12,"Scott Jurek, Steve Friedman",2012.0,Eat And Run,Eat and Run: My Unlikely Journey to Ultramarat...,en-US,3.95,17376,20086,1622,284,857,4776,7873,6296,https://images.gr-assets.com/books/1344746039m...,https://images.gr-assets.com/books/1344746039s...
5157,5158,59819,59819,1961184,56,515128546,9.780515e+12,Nora Roberts,1999.0,Tears of the Moon,Tears of the Moon (Gallaghers of Ardmore / Iri...,en-US,4.19,24613,26184,581,98,652,4882,9226,11326,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [ ]:
books.isna().sum()

id                              0
book_id                         0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

In [ ]:
# drop books without any publication id (isbn, isbn13) or unknown publication year
books.dropna(axis=0, subset=['isbn', 'isbn13'], how='all', inplace=True)
books.dropna(axis=0, subset=['original_publication_year'], inplace=True)

books['language_code'].fillna(value='Unknown', inplace=True)
books['original_title'].fillna(value=books['title'], inplace=True)

In [ ]:
ratings = pd.read_csv('input/ratings.csv', encoding = "ISO-8859-1")
print(ratings.shape, ratings.isna().values.any(), ratings.duplicated().values.any())
print(ratings.columns)
ratings.sample(4)

(981756, 3) False True
Index(['book_id', 'user_id', 'rating'], dtype='object')


,book_id,user_id,rating
538177,5400,1053,5
289180,2894,52673,5
208987,2092,10998,4
598228,6006,49755,5


In [ ]:
ratings.drop_duplicates(keep='first', inplace=True)

In [ ]:
book_tags = pd.read_csv('input/book_tags.csv', encoding = "ISO-8859-1")
print(book_tags.shape, book_tags.isna().values.any(), book_tags.duplicated().values.any())
print(book_tags.columns)
book_tags.sample(4)
# count probably number of people who set those tags on books

(999912, 3) False True
Index(['goodreads_book_id', 'tag_id', 'count'], dtype='object')


,goodreads_book_id,tag_id,count
553097,769712,8717,330
820804,13152668,9920,10
295855,70509,617,5
213676,34506,27401,25


In [ ]:
book_tags.drop_duplicates(subset=['goodreads_book_id', 'tag_id'], keep='first', inplace=True)

In [ ]:
tags = pd.read_csv('input/tags.csv')
print(tags.shape, tags.isna().values.any(), tags.duplicated(subset=['tag_name']).values.any())
print(tags.columns)
tags.sample(4)

(34252, 2) False False
Index(['tag_id', 'tag_name'], dtype='object')


,tag_id,tag_name
12550,12550,freemasons
14840,14840,horror-macabre
3679,3679,award-winning
8291,8291,cormac


In [ ]:
books = books.merge(
    book_tags.merge(tags, how='inner', left_on='tag_id', right_on='tag_id'),
    left_on='book_id', right_on='goodreads_book_id', how='inner' 
)

In [ ]:
grouped_tags = books.groupby(by='book_id')['tag_name'].apply(' '.join).reset_index()
print(grouped_tags.shape)
grouped_tags.head()

(9417, 2)


,book_id,tag_name
0,1,to-read fantasy favorites currently-reading yo...
1,2,to-read fantasy favorites currently-reading yo...
2,3,to-read fantasy favorites currently-reading yo...
3,5,to-read fantasy favorites currently-reading yo...
4,6,to-read fantasy young-adult fiction harry-pott...


In [ ]:
books = pd.merge(
    books.iloc[:, :-3],
    books.groupby(by='book_id')['tag_name'].apply(' '.join).reset_index(),
    left_on='book_id', right_on='book_id', how='inner'
).drop_duplicates(keep='first').reset_index(drop=True)
books['corpus'] = pd.Series(books[['authors', 'tag_name']].values.tolist()).str.join(' ')

## Построение Рекоммендательной системы

1. Приведите данные датасета `ratings` к виду датафрейма со строками-пользователями, столбцами-книгами и рейтингами на пересечении. Будем дополнять информацию о пользователе на основе прочитанных пользователем книг

In [ ]:
full_user_vectors = ratings.pivot_table(values='rating', index='user_id', columns=['book_id']).fillna(0)
user_vectors_index = full_user_vectors.index

csr_user_vectors = csr_matrix(full_user_vectors.values)
del(full_user_vectors)
assert(csr_user_vectors.shape == (53424, 10000))

2. Создайте векторы, характеризующие пользоваталей (для content-based лучше это делать по дополнительным данным о пользователях, но так как тут у нас их нет - сделаем вектора-признаки (размерностью 100) на основе проставленных книгам оценок) с помощью метода понижения размерности PCA до 100.

In [ ]:
# IncrementalPCA builds a lower-rank batch SVDs for the input data, 
# keeping only the most significant singular vectors to project the data to a lower dimensional space
# allows sparse input

# complexity of each SVD (batch_size*n_features ** 2); n_SVD-s n_samples / batch_size 
economy_pca = IncrementalPCA(n_components=100, batch_size=csr_user_vectors.shape[0] // 100)
pca_user_vectors = economy_pca.fit_transform(csr_user_vectors)

tf-idf на основе `books['corpus']` для каждой книги. Корпус документов в виде наборов автор+тэг

In [ ]:
# on top 200 most frequent words
tf_corpus = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), min_df=0, max_features=200, stop_words='english')
tfidf_matrix_corpus = tf_corpus.fit_transform(books['corpus'])

Сохранение результатов перед отвалом, чтобы заново не пересчитывать:

In [ ]:
!mkdir 'tmp'

mkdir: cannot create directory ‘tmp’: File exists


In [ ]:
save_npz('tmp/tfidf_corpus.npz', tfidf_matrix_corpus)
save_npz('tmp/pca_user_vectors.npz', csr_matrix(pca_user_vectors))

pd.Series(user_vectors_index, index=user_vectors_index).to_csv('tmp/user_ids.csv', header=True, index=False, columns=['user_id'])
books.to_csv('tmp/books.csv', index=False)

3. Объедините три датасета:  
    * ratings
    * pca_user_vectors(векторы-признаки для каждого пользователя)
    * tf-idf на основе `books['corpus']` для каждой книги

In [ ]:
from scipy.sparse import load_npz

ratings = pd.read_csv('input/ratings.csv')
user_vectors_index = pd.read_csv('tmp/user_ids.csv')
pca_user_vectors = load_npz('tmp/pca_user_vectors.npz')
tfidf_matrix_corpus = load_npz('tmp/tfidf_corpus.npz')
pca_user_vectors_df = pd.DataFrame(pca_user_vectors.toarray(), index=user_vectors_index['user_id'].values)

books = pd.read_csv('tmp/books.csv')

In [ ]:
corpus_df = pd.DataFrame(data=tfidf_matrix_corpus.toarray(), index=books['book_id'].values)

In [ ]:
data = ratings.merge(corpus_df.add_suffix('_wrd'), left_on='book_id', right_index=True, how='inner', copy=False)
data = data.merge(pca_user_vectors_df.add_suffix('_usftr'), left_on='user_id', right_index=True, how='inner', copy=False)

4. Разбейте на train/valid set



In [ ]:
features = data.columns[3:].to_list()
target = 'rating'

Train, Valid, Test = np.split(
    data[features+[target]].sample(frac=1, random_state=5), 
    [int(.75*len(data)), int(.9*len(data))]
)
X_train, X_valid, X_test = Train[features], Valid[features], Test[features]
y_train, y_valid, y_test = Train[target], Valid[target], Test[target]

5. Обучите любую модель машинного обучения (для задачи регрессии (линейную или ансамбль деревьев)), сделайте прогноз и посчитайте метрики RMSE, MSE


Буду использовать Лассо из-за разреженного характера признакого пространства

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

alphas = np.logspace(-3, 1, 5)

searcher = GridSearchCV(
    estimator=Lasso(), param_grid={'alpha': alphas}, 
    cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error', refit=False
)
searcher.fit(X_valid, y_valid)
best_alpha = searcher.best_params_['alpha']

In [ ]:
model = Lasso(best_alpha)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"MSE: {np.square(y_pred - y_test).sum() / y_test.shape[0] : .4f}, with Lasso({best_alpha:.2f})")

MSE:  0.9512, with Lasso(0.01)


6. Добавьте другие признаки по книгам из `books`

In [ ]:
numeric_features = ['books_count', 'work_ratings_count', 'work_text_reviews_count']
categorical_features = ['language_code', 'original_publication_year']

data = data.merge(books[numeric_features + categorical_features + ['book_id']], on='book_id', how='inner')

In [ ]:
data.loc[:, 'original_publication_year'] -= data['original_publication_year'] % 10
data['original_publication_year'] = data['original_publication_year'].apply(str)
data['original_publication_year'].nunique()

35

In [ ]:
numeric_features += features
updated_features = numeric_features+categorical_features

Train, Valid, Test = np.split(
    data[updated_features+[target]].sample(frac=1, random_state=5), 
    [int(.75*len(data)), int(.9*len(data))]
)

X_train, X_valid, X_test = Train[updated_features], Valid[updated_features], Test[updated_features]
y_train, y_valid, y_test = Train[target], Valid[target], Test[target]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

trfrm = ColumnTransformer(
    [
     ('numerical', StandardScaler(), numeric_features), 
     ('categorical', OneHotEncoder(), categorical_features)
     ]
)

searcher = GridSearchCV(
    estimator=make_pipeline(trfrm, Lasso()), param_grid={'lasso__alpha': alphas},
    cv=3, n_jobs=-1, scoring='neg_root_mean_squared_error', refit=False
)
searcher.fit(X_valid, y_valid)
best_alpha = searcher.best_params_['lasso__alpha']

In [ ]:
pipe_model = make_pipeline(trfrm, Lasso(best_alpha))
pipe_model.fit(X_train, y_train)
y_pred = pipe_model.predict(X_test)

print(f"MSE: {np.square(y_pred - y_test).sum() / y_test.shape[0] : .4f}, with Lasso({best_alpha:.2f})")

MSE:  0.9330, with Lasso(0.01)


7. Сделайте тестовый датасет для пользователей и книг, которые находятся в датасете to-read:
    * pca_user_vectors(векторы-признаки для каждого пользователя)
    * tf-idf на основе `books['corpus']` для каждой книги
8. Сделайте для них прогноз

In [ ]:
to_read_df = pd.read_csv('input/to_read.csv')
print(to_read_df.shape, to_read_df.isna().values.any(), to_read_df.duplicated().values.any())
print(to_read_df.columns)
to_read_df.sample(4)

(912705, 2) False False
Index(['user_id', 'book_id'], dtype='object')


,user_id,book_id
157528,11404,211
187668,13212,984
580690,34556,178
700867,41149,1508


In [ ]:
test_data = to_read_df.merge(corpus_df.add_suffix('_wrd'), left_on='book_id', right_index=True, how='inner', copy=False)
test_data = test_data.merge(pca_user_vectors_df.add_suffix('_usftr'), left_on='user_id', right_index=True, how='inner', copy=False)
test_data = test_data.merge(books[list(set(updated_features) ^ set(features))+['book_id']], on='book_id', how='inner', copy=False)
test_data.dtypes

user_id                        int64
book_id                        int64
0_wrd                        float64
1_wrd                        float64
2_wrd                        float64
                              ...   
language_code                 object
books_count                    int64
original_publication_year    float64
work_ratings_count             int64
work_text_reviews_count        int64
Length: 307, dtype: object

In [ ]:
test_data.loc[:, 'original_publication_year'] -= test_data['original_publication_year'] % 10
test_data['original_publication_year'] = test_data['original_publication_year'].apply(str)
test_pred_ratings = pipe_model.predict(test_data[updated_features])

In [ ]:
res = pd.concat([test_data[['user_id', 'book_id']], pd.Series(data=test_pred_ratings).rename("rating_pred")], axis=1)
res.to_csv('input/result_to_read.csv', index=False)
res.head(8)

,user_id,book_id,rating_pred
0,1,112,3.806865
1,9,112,3.801736
2,72,112,3.806566
3,101,112,3.809787
4,164,112,3.786541
5,241,112,3.806404
6,292,112,3.808775
7,431,112,3.807736


## Вывод ## 
Были построены рекоммендации для книг сервиса `goodreads`, по имеющимся оценкам пользователей для тех же книг с других ресурсов. В процессе решения были использованы методы отбора признаков путем понижения размерности и решена задача регрессии через Lasso (линейная регрессия + l1 регуляризация) с подбором оптимального коэф-та регуляризации. Был выбран именно этот метод, т.к. имелось большое признаковое пространство.